In [23]:
from base.base_inference import VideoInference
from models import UNet
from models import DeepLabV3Plus
from subprocess import call
import torch
import ffmpeg

In [24]:
CHECKPOINT = "./pretr/DeepLabV3Plus_ResNet18.pth"
BACKBONE   = "resnet18"
VIDEO_INP  = "./vd/bar.mp4"
VIDEO_OUT  = "./vd/out.mp4"
VIDEO_TMP  = "./vd/.tmp.mp4"

In [15]:
!nvidia-smi

Tue Mar 24 10:43:15 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0  On |                  N/A |
| 68%   76C    P2   196W / 250W |  10952MiB / 10989MiB |     83%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 30%   49C    P8    10W / 250W |    765MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [16]:
torch.cuda.set_device(1)
torch.cuda.current_device()

1

In [25]:
model = DeepLabV3Plus(backbone=BACKBONE, num_classes=2)
trained_dict = torch.load(CHECKPOINT, map_location="cpu")['state_dict']
model.load_state_dict(trained_dict, strict=False)
model.cuda()
model.eval()
print('ok')

[ResNet] Initialize weights...
ok


In [27]:
call(["rm", VIDEO_TMP])
inference = VideoInference(
    model=model,
    video_path=VIDEO_INP,
    video_out_path=VIDEO_TMP,
    input_size=480,
    background_path = "./backgrounds/white.jpg",
    use_cuda=True,
    draw_mode='matting',
    frame_range=(0, 1000)
)
inference.run()

100% (1000 of 1000) |####################| Elapsed Time: 0:00:20 Time:  0:00:20


In [16]:
?ffmpeg.output

In [28]:
in1 = ffmpeg.input(VIDEO_INP)
in2 = ffmpeg.input(VIDEO_TMP)
out = ffmpeg.output(in1.audio, in2.video, VIDEO_OUT)
out.run(overwrite_output=True)

(None, None)

In [29]:
from IPython.display import Video
Video(VIDEO_OUT, width = 350, height = 350)